In [3]:
from arcgis.gis import GIS
from arcgis.features import FeatureLayer
import requests
import os
import pandas as pd
import math


username = "GWPConsultants"
password = "Uptonhouse*3124"

gis = GIS(username=username, password=password)

In [4]:
# Use the webmap here to get the layer dataframe
data_item_id = 'aab6bf9c574b41eebb8ce532c21bc826'

webmap_item = gis.content.get(data_item_id)

webmap_data = webmap_item.get_data()

webmap_layers = webmap_data['operationalLayers']

In [5]:
# Get the feature layer from the portal or something
FeatureLayerID = "40f4bbc9370541188cd7a6adb7f65551"
# Set some destination
save_folder = "P:/Tools/LaTex test/"
print(save_folder)

P:/Tools/LaTex test/


In [6]:
# This is just assuming the first layer is what we want. Can be searchd for better
first_layer_url = webmap_layers[0]['url']
feature_layer = FeatureLayer(first_layer_url)

features = feature_layer.query(where="1=1", return_geometry=False).features

if features:
    data = []
    for feature in features:
        feature_attributes = feature.attributes
        data.append(feature_attributes)
        
    df = pd.DataFrame(data)
    df = df.applymap(lambda x: str(x).replace('_', ' '))
    
else:
    print("No features found.")

In [29]:
FeatureLayer = gis.content.get(FeatureLayerID)
layer = FeatureLayer.layers[0]

existing_oids = [f.attributes['OBJECTID'] for f in layer.query(where="1=1").features]

oid_attid_dict = {} 

attid = 1
for oid in existing_oids:
    attachments_list = layer.attachments.get_list(oid=oid)
    oid_attid_dict[int(oid)] = attid  # Convert oid to integer here
    attid += 1

print("OID to AttID mapping:", oid_attid_dict)

for oid, attid in oid_attid_dict.items():
    name = df.loc[df['OBJECTID'] == str(oid), 'PTName'].values[0] + '.jpg'
    layer.attachments.download(oid=oid, attachment_id=attid, save_path=f"{save_folder}")
    os.rename(os.path.join(save_folder, 'Photo 1.jpg'), os.path.join(save_folder, name))
    
    
    
# for oid, attid in oid_attid_dict.items():
#     layer.attachments.download(oid=oid, attachment_id=attid, save_path=f"{save_folder}")
#     filt = df['OBJECTID'] == oid
#     name = df[filt]['PTName'][l] + '.jpg'
#     os.rename(os.path.join(save_folder, 'Photo 1.jpg'), os.path.join(save_folder, name))
#     l += 1

OID to AttID mapping: {1: 1, 3: 2, 4: 3}


In [31]:
#This is used to make the tex file. Uses a template and just chucks stuff in afterwards. Finishes off by adding an \end document
base_template = r"""
\documentclass[12pt, letterpaper]{article}
\usepackage{graphicx}
\graphicspath{{#SAVE_FOLDER#}}
\usepackage{subcaption}
\usepackage[left=10mm,right=10mm,top=5mm,bottom=5mm,paper=a4paper]{geometry}
\usepackage{multirow}
\usepackage{caption}  % Added for \captionof
\usepackage{xcolor}
\usepackage{fontspec}
\setmainfont{Tahoma}
\usepackage[utf8]{inputenc}

\setlength{\fboxsep}{0pt} % <-- Add this line
\captionsetup[subfigure]{labelformat=empty}

\begin{document}
"""

# Define the section template to be added
section_template = r"""
\begin{figure}[h]
	\centering
	\fbox{\parbox{1\linewidth}{
	\vspace{5mm}
			\begin{subfigure}{\linewidth}
				\centering
				\includegraphics[height=0.37\textheight]{#photopath1#}
				\caption{#CAPTION1# #title1# - #comment1#}
				%\label{fig:dragon1}
			\end{subfigure}

			\vspace{\baselineskip}

            \begin{subfigure}{\linewidth}
				\centering
				\includegraphics[height=0.37\textheight]{#photopath2#}
				\caption{#CAPTION2# #title2# - #comment2#}
				%\label{fig:dragon1}
			\end{subfigure}

			\vspace{\baselineskip}

			\hrule
			 \begin{minipage}[c][3.5cm][t]{0.4\textwidth} % Larger Left side
			        %\centering
				\begin{minipage}[t][1.5cm][t]{0.4\textwidth} % Set explicit width for Right top
				\vspace{2mm}
				\hspace{2mm}
				    \raggedright
				    \textbf{\small{Notes:} \footnotesize{}}

				\end{minipage}
				 \hrule
			            \begin{minipage}[c][2cm][c]{\textwidth} % Adjust height and alignment
				    \centering
			                \includegraphics[height=1.8cm]{P:/Tools/DocumentGeneration/gwp.png}
			            \end{minipage}
			    \end{minipage}%
			    \vline % Vertical line
			    \begin{minipage}[c][3cm][t]{0.6\textwidth} % Larger Right side with no indentation
			        \begin{minipage}[c][1cm][t]{0.6\textwidth} % Set explicit width for Right top
				\vspace{2mm}     
				\hspace{2mm}
				    \raggedright
				\textbf{\small{Client:} \footnotesize{}} 
			        \end{minipage}
			        \hrule % Horizontal line
			        \begin{minipage}[c][1.4cm][t]{0.6\textwidth} % Set explicit width for Right bottom
				\vspace{2mm}
				\hspace{2mm}
				    \raggedright
				 \textbf{\small{Project:} \footnotesize{}} 
			        \end{minipage}
			        \hrule % Horizontal line
			        \begin{minipage}[c][0.6cm][t]{0.6\textwidth} % Set explicit width for Right bottom    
				\vspace{2mm}
				\hspace{2mm}
				    \raggedright
				\textbf{\footnotesize Appendix:} 
			        \end{minipage}
			    \end{minipage}
			
		}}
\end{figure}
"""

# --------------------------------------------------------------------------------------------------------------------
result = math.ceil(len(df) / 2)

df.sort_values(by='OBJECTID')
result_df = df[['PTName', 'Comment', 'OBJECTID']]
result_df['photonum'] = range(1, len(result_df) + 1)

# Check if the number of images is odd
if len(result_df) % 2 == 1:
    # Duplicate the last row
    last_row = result_df.iloc[-1].copy()
    last_row['OBJECTID'] = str(int(last_row['OBJECTID']) + 1)
    last_row['photonum'] += 1
    result_df = pd.concat([result_df, pd.DataFrame([last_row])], ignore_index=True)
# ////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

# //////////////////////////////////////////////////////////////////////////////////////////////////////////////
# Build the TeX content by replacing # LOOP_SECTION # with the section template within a loop
photo=1

tex_content = base_template
for i in range(result):
    caption1 = f"Photo {photo}:"
    caption2 = f"Photo {photo+1}:"

    filtered=result_df['photonum'] == photo
    title1 = result_df.loc[filtered, 'PTName'].iloc[0]
    comment1 = result_df.loc[filtered, 'Comment'].iloc[0]
    photopath1 = (result_df.loc[filtered, 'PTName'].iloc[0] + '.jpg')
    photo+=1

    filtered2=result_df['photonum'] == photo
    title2 = result_df.loc[filtered2, 'PTName'].iloc[0]
    comment2 = result_df.loc[filtered2, 'Comment'].iloc[0]
    photopath2 = (result_df.loc[filtered2, 'PTName'].iloc[0] + '.jpg')
    photo+=1

    section_content = section_template.replace("#CAPTION1#", caption1).replace("#photopath1#", photopath1).replace("#title1#", str(title1)).replace("#comment1#", str(comment1)).replace("#CAPTION2#", caption2).replace("#photopath2#", photopath2).replace("#title2#", str(title2)).replace("#comment2#", str(comment2))
    tex_content += section_content

tex_content += r"\end{document}"

tex_content = tex_content.replace("#SAVE_FOLDER#", save_folder)

output_filename = "output.tex"
output_path = os.path.join(save_folder, output_filename)

with open(output_path, "w") as tex_file:
    tex_file.write(tex_content)

[31]:94: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
